In [2]:
import pandas as pd
import numpy as np
import os
import glob
import xml.etree.ElementTree as ET
import csv

Segment into pre-data and post-data

#### Importing XML files

In [2]:
def import_excel_xml_data_from_folder(folder_path):
    """
    Import Excel 2003 XML (SpreadsheetML) data from all .xml files in a folder
    into one consolidated DataFrame.
    """
    xml_files = glob.glob(os.path.join(folder_path, "*.xml"))
    if not xml_files:
        raise FileNotFoundError(f"No XML files found in {folder_path!r}")
    dfs = []
    # Namespace for SpreadsheetML
    ns = {'ss': 'urn:schemas-microsoft-com:office:spreadsheet'}
    for file_path in xml_files:
        tree = ET.parse(file_path)
        root = tree.getroot()
        # Locate the Table element
        table = root.find(".//ss:Table", ns)
        if table is None:
            print(f"  → no <Table> found in {file_path}, skipping.")
            continue
        rows = table.findall("ss:Row", ns)
        if not rows:
            print(f"  → no <Row> entries in {file_path}, skipping.")
            continue
        # 1) Extract headers from the first row
        header_cells = rows[0].findall("ss:Cell", ns)
        headers = [
            cell.find("ss:Data", ns).text 
            for cell in header_cells
        ]
        # 2) Extract every subsequent row
        data = []
        for row in rows[1:]:
            cells = row.findall("ss:Cell", ns)
            row_vals = []
            for i in range(len(headers)):
                try:
                    data_elem = cells[i].find("ss:Data", ns)
                    row_vals.append(data_elem.text if data_elem is not None else None)
                except IndexError:
                    row_vals.append(None)
            data.append(row_vals)
        df = pd.DataFrame(data, columns=headers)
        df['source_file'] = os.path.basename(file_path)
        dfs.append(df)
        print(f"  → imported {df.shape[0]} rows from {os.path.basename(file_path)}")
    if not dfs:
        raise ValueError("No data was successfully imported from any file.")
    combined = pd.concat(dfs, ignore_index=True)
    print(f"Combined DataFrame: {combined.shape[0]} rows × {combined.shape[1]} columns")
    return combined

In [3]:
amb_folder = 'AMB'
amb_df = import_excel_xml_data_from_folder(amb_folder)
amb_df.to_parquet('amb_data.parquet')

  → imported 64380 rows from SignalDownload_23-12-December.xml
  → imported 63635 rows from SignalDownload_24-01-January.xml
  → imported 66281 rows from SignalDownload_24-02-February.xml
  → imported 69387 rows from SignalDownload_24-03-March.xml
  → imported 66775 rows from SignalDownload_24-04-April.xml
  → imported 67195 rows from SignalDownload_24-05-May.xml
  → imported 66412 rows from SignalDownload_24-06-June.xml
  → imported 65723 rows from SignalDownload_24-07-July.xml
  → imported 70478 rows from SignalDownload_24-08-August.xml
  → imported 71733 rows from SignalDownload_24-09-September.xml
  → imported 70172 rows from SignalDownload_24-10-October.xml
  → imported 73078 rows from SignalDownload_24-11-November.xml
  → imported 68910 rows from SignalDownload_24-12-December.xml
  → imported 68465 rows from SignalDownload_25-01-January.xml
  → imported 68573 rows from SignalDownload_25-02-February.xml
  → imported 72923 rows from SignalDownload_25-03-March.xml
  → imported 70545

In [4]:
ip_folder = 'IP'
ip_df = import_excel_xml_data_from_folder(ip_folder)
ip_df.to_parquet('ip_data.parquet')

  → imported 89980 rows from SignalDownload_23-12-December.xml
  → imported 84156 rows from SignalDownload_24-01-January.xml
  → imported 87684 rows from SignalDownload_24-02-February.xml
  → imported 90051 rows from SignalDownload_24-03-March.xml
  → imported 89880 rows from SignalDownload_24-04-April.xml
  → imported 91038 rows from SignalDownload_24-05-May.xml
  → imported 87909 rows from SignalDownload_24-06-June.xml
  → imported 87340 rows from SignalDownload_24-07-July.xml
  → imported 96752 rows from SignalDownload_24-08-August.xml
  → imported 90980 rows from SignalDownload_24-09-September.xml
  → imported 94046 rows from SignalDownload_24-10-October.xml
  → imported 102223 rows from SignalDownload_24-11-November.xml
  → imported 94361 rows from SignalDownload_24-12-December.xml
  → imported 93510 rows from SignalDownload_25-01-January.xml
  → imported 94918 rows from SignalDownload_25-02-February.xml
  → imported 110842 rows from SignalDownload_25-03-March.xml
  → imported 920

#### Import previously-saved parquet files

In [3]:
amb_df = pd.read_parquet('amb_data.parquet')
ip_df = pd.read_parquet('ip_data.parquet')

In [4]:
amb_df.Value = pd.to_numeric(amb_df.Value)
ip_df.Value = pd.to_numeric(ip_df.Value)

Limit to users related to a particular optimization

In [6]:
#list of all providers
ip_provs = ip_df['Clinician Name'].drop_duplicates().sort_values()

with open('ip_provs.csv', 'w', newline='') as file:
    # Don't use csv.writer since we want to manually format each line
    for item in list(set(ip_provs)):
        # Format each line with quotes around the item and a comma at the end
        file.write(f'"{item}",\n')

In [7]:
prov = ip_df[ip_df['User Type'] != 'Residents and Fellows'][['Clinician Name','Login Department', 'Specialty', 'User Type']].drop_duplicates()
prov

,Clinician Name,Login Department,Specialty,User Type
0,"Dassinger, Melvin Sidney III, MD",NW SURGERY CL,General Surgery,Physician
174,"Agarwal, Amit, MD",ACH PULMONARY CL,Pediatric Pulmonology,Physician
385,"Albert, Gregory W., MD",ACH NEUROSURGERY CL,Neurosurgery,Physician
565,"Angtuaco, Michael J., MD",ACH CARDIOLOGY CL,Cardiology,Physician
761,"Appell, Lauren, MD",ACH BONE MARROW TRANSP,Hematology and Oncology,Physician
...,...,...,...,...
1540741,"Schmitz, Michael L, MD",ACH ANESTHESIOLOGY,Anesthesiology,Physician
1544288,"Garcia, Xiomara, MD",ACH HEART TRANSPLANT,Transplant,Physician
1544989,"Thompson, Tonya M, MD",ACH EMERGENCY,Sedation,Physician
1553708,"Goodman, Allyson, MD",NW EMERGENCY,Emergency Medicine,Physician


In [8]:
hosp = prov[
    ((prov['Login Department'] == 'ACH IP HOSPITALIST') | (prov['Clinician Name'].str.contains('Bolden')) | (prov['Clinician Name'].str.contains('Barkley')) | (prov['Clinician Name'].str.contains('Tilly'))) 
    &
    (prov['User Type'] != 'APP') 
    &
    ~((prov['Clinician Name'].str.contains('Farrar')) | (prov['Clinician Name'].str.contains('Barr')) | (prov['Clinician Name'].str.contains('Latch')) | (prov['Clinician Name'].str.contains('Nguyen')))
].sort_values(by='Clinician Name')

In [9]:
hosp

,Clinician Name,Login Department,Specialty,User Type
27938,"Alkhatib, Lynn, MD",ACH IP HOSPITALIST,Pediatrics,Physician
37741,"Barkley, Hannah G, MD",ACH COF TEACHING CL,Pediatrics,Physician
8196,"Barnes, Emily S, MD",ACH IP HOSPITALIST,Pediatrics,Physician
24938,"Barnes, Sarah A., MD",ACH IP HOSPITALIST,Pediatrics,Physician
70613,"Bolden, Branson B, MD",ACH AFTER HOURS CL,Internal Medicine,Physician
40658,"Cantu, Rebecca, MD",ACH IP HOSPITALIST,Pediatrics,Physician
34444,"Cook, Devin W, DO",ACH IP HOSPITALIST,Pediatrics,Physician
34159,"Dailey, Raymond Lane, MD",ACH IP HOSPITALIST,Pediatrics,Physician
66621,"Ekdahl, Rachel, MD",ACH IP HOSPITALIST,Pediatrics,Physician
45010,"Filipek, Jacob F, MD",ACH IP HOSPITALIST,Pediatrics,Physician


In [10]:
#identify list of providers in optmization
inc_provs = list(set(hosp['Clinician Name']))

In [11]:
#list of all signal metrics
ip_mets = ip_df['Metric'].drop_duplicates().sort_values()
amb_mets = amb_df['Metric'].drop_duplicates().sort_values()

with open('ip_metrics.csv', 'w', newline='') as file:
    # Don't use csv.writer since we want to manually format each line
    for item in list(set(ip_mets)):
        # Format each line with quotes around the item and a comma at the end
        file.write(f'"{item}",\n')

with open('amb_metrics.csv', 'w', newline='') as file:
    # Don't use csv.writer since we want to manually format each line
    for item in list(set(amb_mets)):
        # Format each line with quotes around the item and a comma at the end
        file.write(f'"{item}",\n')

In [12]:
ip_mets = ip_df['Metric'].drop_duplicates().sort_values()

In [13]:
#enter list of hypothesized metrics 
hypo_metrics = ['']

In [14]:
#limit data to those in the optimization group
opt_data_ip = ip_df[ip_df['Clinician Name'].isin(inc_provs)]
opt_data_amb = amb_df[amb_df['Clinician Name'].isin(inc_provs)]

shared_col = ['EMP CID', 'SER CID', 'Clinician Name', 'Clinician Type', 'Specialty', 'User Type',
       'Reporting Period Start Date', 'Reporting Period End Date', 'Metric',
       'Numerator', 'Denominator', 'Value', 'Metric ID', 'source_file']

opt_data = pd.concat([opt_data_ip, opt_data_amb], ignore_index=False)

Segment into pre-data and post-data

Before/After comparisons in % differences

In [15]:
opt_data['Reporting Period Start Date'] = pd.to_datetime(opt_data['Reporting Period Start Date'])
opt_data['Reporting Period End Date'] = pd.to_datetime(opt_data['Reporting Period End Date'])

In [16]:
opt_data[['Reporting Period Start Date','Reporting Period End Date']].drop_duplicates().sort_values(by='Reporting Period Start Date')

,Reporting Period Start Date,Reporting Period End Date
3598,2023-11-26,2023-12-30
93695,2023-12-31,2024-01-27
177701,2024-01-28,2024-02-24
268856,2024-02-25,2024-03-30
355944,2024-03-31,2024-04-27
445651,2024-04-28,2024-05-25
536115,2024-05-26,2024-06-29
624170,2024-06-30,2024-07-27
712149,2024-07-28,2024-08-31
808545,2024-09-01,2024-09-28


In [17]:
pre_start = pd.to_datetime('2023-11-26')
pre_end = pd.to_datetime('2024-02-25')
post_start = pd.to_datetime('2024-09-29')
post_end = pd.to_datetime('2024-12-29')

In [18]:
predata = opt_data[(opt_data['Reporting Period Start Date'] >= pre_start) & (opt_data['Reporting Period Start Date'] <= pre_end)]
predata['Period'] = 'Pre'
postdata = opt_data[(opt_data['Reporting Period Start Date'] >= post_start) & (opt_data['Reporting Period Start Date'] <= post_end)]
postdata['Period'] = 'Post'

/tmp/ipykernel_1155/823114164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predata['Period'] = 'Pre'
/tmp/ipykernel_1155/823114164.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postdata['Period'] = 'Post'


Section level

In [19]:
predata

,EMP CID,SER CID,Clinician Name,Clinician Type,Login Service Area,Login Department,Specialty,User Type,Reporting Period Start Date,Reporting Period End Date,Metric,Numerator,Denominator,Value,Metric ID,source_file,Service Area,Department,Period
3598,1705995,1408856,"Liu, Daniel S, MD",PHYSICIAN,ACH SERVICE AREA,ACH IP HOSPITALIST,Pediatrics,Physician,2023-11-26,2023-12-30,(IP) Time in Notes per Day,425.98329299999995,20,21.299164649999998,2119,SignalDownload_23-12-December.xml,NaN,NaN,Pre
3599,1705995,1408856,"Liu, Daniel S, MD",PHYSICIAN,ACH SERVICE AREA,ACH IP HOSPITALIST,Pediatrics,Physician,2023-11-26,2023-12-30,(IP) Time in Notes per Patient per Day,425.98329299999995,210,2.0284918714285713,2120,SignalDownload_23-12-December.xml,NaN,NaN,Pre
3600,1705995,1408856,"Liu, Daniel S, MD",PHYSICIAN,ACH SERVICE AREA,ACH IP HOSPITALIST,Pediatrics,Physician,2023-11-26,2023-12-30,(IP) Note Composition Method - Manual,18231,126150,0.14451843043995244,2121,SignalDownload_23-12-December.xml,NaN,NaN,Pre
3601,1705995,1408856,"Liu, Daniel S, MD",PHYSICIAN,ACH SERVICE AREA,ACH IP HOSPITALIST,Pediatrics,Physician,2023-11-26,2023-12-30,(IP) Note Composition Method - Voice Recognition,702,126150,0.005564803804994055,2121,SignalDownload_23-12-December.xml,NaN,NaN,Pre
3602,1705995,1408856,"Liu, Daniel S, MD",PHYSICIAN,ACH SERVICE AREA,ACH IP HOSPITALIST,Pediatrics,Physician,2023-11-26,2023-12-30,(IP) Note Composition Method - Transcription,0,126150,0,2121,SignalDownload_23-12-December.xml,NaN,NaN,Pre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260257,17015488,17014423,"Barkley, Hannah G, MD",PHYSICIAN,NaN,NaN,Pediatrics,Physician,2024-02-25,2024-03-30,Secure Chat Messages Sent per Appointment,660,9,73.33333333333333,1153,SignalDownload_24-03-March.xml,ACH SERVICE AREA,ACH COF TEACHING CL,Pre
260258,17015488,17014423,"Barkley, Hannah G, MD",PHYSICIAN,NaN,NaN,Pediatrics,Physician,2024-02-25,2024-03-30,Secure Chat Patient Conversations,28,36,0.7777777777777778,1156,SignalDownload_24-03-March.xml,ACH SERVICE AREA,ACH COF TEACHING CL,Pre
260259,17015488,17014423,"Barkley, Hannah G, MD",PHYSICIAN,NaN,NaN,Pediatrics,Physician,2024-02-25,2024-03-30,Secure Chat Messages Received per Day,1175,1,1175,1249,SignalDownload_24-03-March.xml,ACH SERVICE AREA,ACH COF TEACHING CL,Pre
260260,17015488,17014423,"Barkley, Hannah G, MD",PHYSICIAN,NaN,NaN,Pediatrics,Physician,2024-02-25,2024-03-30,Secure Chat Messages Received per Appointment,1175,9,130.55555555555554,1250,SignalDownload_24-03-March.xml,ACH SERVICE AREA,ACH COF TEACHING CL,Pre


In [20]:
mets = predata[['Metric', 'Value']].sort_values(by='Metric')

In [21]:
mets.shape

(16200, 2)

In [22]:
pre_med = pd.DataFrame(predata[['Metric', 'Value']].groupby('Metric').agg(['median']))
pre_med.columns = ['Pre Med Value']
post_med = pd.DataFrame(postdata[['Metric', 'Value']].groupby('Metric').agg(['median']))
post_med.columns = ['Post Med Value']

TypeError: agg function failed [how->median,dtype->object]

In [ ]:
pre_post = pre_med.join(post_med)

In [ ]:
pre_post

In [ ]:
pre_post['PercentDiff'] = (pre_post['Post Med Value'] - pre_post['Pre Med Value'])/pre_post['Pre Med Value']
pre_post['AbsDiff'] = (pre_post['Post Med Value'] - pre_post['Pre Med Value'])

Data Driven - Identify percent changes > 5%

In [ ]:
sig_diff = pre_post[(pre_post['PercentDiff'] <= -0.05) | (pre_post['PercentDiff'] >= 0.05)]

In [ ]:
sig_diff_metrics = list(sig_diff.index.drop_duplicates())

Hypothesis Driven

In [ ]:
pre_post[pre_post.Metric.isin(hypo_metrics)]

Final Metrics to include in Report